In [31]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import os.path
import time

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import cifar10

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('train_dir', '/tmp/cifar10_train',
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 1000000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")


def train():
    """Train CIFAR-10 for a number of steps."""
    with tf.Graph().as_default():
        global_step = tf.Variable(0, trainable=False)

        # Get images and labels for CIFAR-10.
        images, labels = cifar10.distorted_inputs()

        # Build a Graph that computes the logits predictions from the
        # inference model.
        logits = cifar10.inference(images)

        # Calculate loss.
        loss = cifar10.loss(logits, labels)

        # Build a Graph that trains the model with one batch of examples and
        # updates the model parameters.
        train_op = cifar10.train(loss, global_step)

        # Create a saver.
        saver = tf.train.Saver(tf.all_variables())

        # Build the summary operation based on the TF collection of Summaries.
        summary_op = tf.merge_all_summaries()

        # Build an initialization operation to run below.
        init = tf.initialize_all_variables()

        # Start running operations on the Graph.
        sess = tf.Session(config=tf.ConfigProto(
            log_device_placement=FLAGS.log_device_placement))
        sess.run(init)

        # Start the queue runners.
        tf.train.start_queue_runners(sess=sess)

        summary_writer = tf.train.SummaryWriter(FLAGS.train_dir, sess.graph)

        for step in xrange(FLAGS.max_steps):
            start_time = time.time()
            _, loss_value = sess.run([train_op, loss])
            duration = time.time() - start_time

            assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

            if step % 100 == 0:
                num_examples_per_step = FLAGS.batch_size
                examples_per_sec = num_examples_per_step / duration
                sec_per_batch = float(duration)

                format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                              'sec/batch)')
                print (format_str % (datetime.now(), step, loss_value,
                                     examples_per_sec, sec_per_batch))

                summary_str = sess.run(summary_op)
                summary_writer.add_summary(summary_str, step)

            # Save the model checkpoint periodically.
            if step % 1000 == 0 or (step + 1) == FLAGS.max_steps:
                checkpoint_path = os.path.join(FLAGS.train_dir, 'model.ckpt')
                saver.save(sess, checkpoint_path, global_step=step)
            if step % 10 ==0:
                #eval
                xx,yy=cifar10.inputs(eval_data=True)
                xx=cifar10.inference(xx)
                top_k_op=tf.nn.in_top_k(xx,yy,1)
                variable_averages = tf.train.ExponentialMovingAverage(
                cifar10.MOVING_AVERAGE_DECAY)
                variables_to_restore = variable_averages.variables_to_restore()
                saver = tf.train.Saver(variables_to_restore)
                # Build the summary operation based on the TF collection of Summaries.
                summary_op = tf.merge_all_summaries()

                ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
                # Restores from checkpoint
                saver.restore(sess, ckpt.model_checkpoint_path)

                # Start the queue runners.
                coord = tf.train.Coordinator()

                num_iter = int(math.ceil(FLAGS.num_examples / 256))
                true_count = 0  # Counts the number of correct predictions.
                total_sample_count = num_iter * 256
                step = 0
                while step < num_iter and not coord.should_stop():
                    predictions = sess.run([top_k_op])
                    true_count += np.sum(predictions)
                    step += 1

                # Compute precision @ 1.
                precision = true_count / total_sample_count
                print('precision @ 1 = %.3f' % (precision))



                

ArgumentError: argument --train_dir: conflicting option string(s): --train_dir

In [26]:
def main(argv=None):  # pylint: disable=unused-argument
    cifar10.maybe_download_and_extract()
#  if tf.gfile.Exists(FLAGS.train_dir):
#    tf.gfile.DeleteRecursively(FLAGS.train_dir)
#  tf.gfile.MakeDirs(FLAGS.train_dir)
  train()


if __name__ == '__main__':
    tf.app.run()

checkpoint                          model.ckpt-3000       model.ckpt-5000
events.out.tfevents.1461991839.eli  model.ckpt-3000.meta  model.ckpt-5000.meta
model.ckpt-2000                     model.ckpt-4000       model.ckpt-6000
model.ckpt-2000.meta                model.ckpt-4000.meta  model.ckpt-6000.meta


In [32]:
from tensorflow.contrib import skflow
skflow.monitors.

In [33]:
from sklearn import metrics
metrics.accuracy_score